In [165]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score  # Use appropriate metrics for your task


ModuleNotFoundError: No module named 'tensorflow.python'

Dataset

In [90]:
data = {}
import json
with open('nvdcve-1.1-2023.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

#print(data)

In [159]:
def severity2score(str):
    if str == 'NONE'or str == '':
        return 0
    elif str == 'LOW':
        return 1
    elif str == 'MEDIUM':
        return 2
    elif str == 'HIGH':
        return 3
    elif str == 'CRITICAL':
        return 4


cve_items = data.get("CVE_Items", [])

# Create an empty list to store dictionaries representing entries
entries = []

# Extract and append the dictionaries representing entries
for item in cve_items:
    entry = {
        "ID": item['cve']['CVE_data_meta']['ID'],
        "Problem Type": item['cve']['problemtype']['problemtype_data'][0]['description'],
        "CVSS Version": item['impact']['baseMetricV3']['cvssV3']['version'] if 'impact' in item and 'baseMetricV3' in item['impact'] else "",
        "CVSS Vector": item['impact']['baseMetricV3']['cvssV3']['vectorString'] if 'impact' in item and 'baseMetricV3' in item['impact'] else "",
        "CVSS Base Score": float(item['impact']['baseMetricV3']['cvssV3']['baseScore'] if 'impact' in item and 'baseMetricV3' in item['impact'] else "0"),
        "CVSS Severity":  severity2score(item['impact']['baseMetricV3']['cvssV3']['baseSeverity'] if 'impact' in item and 'baseMetricV3' in item['impact'] else ""),
        "Description": item['cve']['description']['description_data'][0]['value'],
        "Config": item['configurations'],
        "Reference": item['cve']['references'],
    }
    entries.append(entry)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(entries)

# Calculate the shape of the DataFrame
num_entries, num_features = df.shape
# Save the DataFrame as a CSV file
df.to_csv("cve_data.csv", index=False)

print("CSV file 'cve_data.csv' saved successfully.")

CSV file 'cve_data.csv' saved successfully.


In [160]:
df.head()

,Problem Type,CVSS Version,CVSS Vector,CVSS Base Score,CVSS Severity,Description,Config,Reference
0,"[{'lang': 'en', 'value': 'CWE-319'}]",3.1,CVSS:3.1/AV:L/AC:L/PR:H/UI:N/S:U/C:H/I:H/A:H,6.7,2,An information exposure vulnerability in the P...,"{'CVE_data_version': '4.0', 'nodes': [{'operat...",{'reference_data': [{'url': 'https://security....
1,"[{'lang': 'en', 'value': 'NVD-CWE-Other'}]",3.1,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H,7.8,3,A problem with a protection mechanism in the P...,"{'CVE_data_version': '4.0', 'nodes': [{'operat...",{'reference_data': [{'url': 'https://security....
2,"[{'lang': 'en', 'value': 'CWE-610'}]",3.1,CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:H/I:N/A:N,6.5,2,A file disclosure vulnerability in the Palo Al...,"{'CVE_data_version': '4.0', 'nodes': [{'operat...",{'reference_data': [{'url': 'https://security....
3,"[{'lang': 'en', 'value': 'NVD-CWE-noinfo'}]",3.1,CVSS:3.1/AV:N/AC:L/PR:H/UI:N/S:U/C:N/I:H/A:H,6.5,2,A local file deletion vulnerability in Palo Al...,"{'CVE_data_version': '4.0', 'nodes': [{'operat...",{'reference_data': [{'url': 'https://security....
4,"[{'lang': 'en', 'value': 'CWE-312'}]",3.1,CVSS:3.1/AV:N/AC:L/PR:H/UI:N/S:U/C:H/I:N/A:N,4.9,2,A vulnerability in Palo Alto Networks PAN-OS s...,"{'CVE_data_version': '4.0', 'nodes': [{'operat...",{'reference_data': [{'url': 'https://security....


In [161]:
df.shape

(14881, 8)

In [ ]:
X = df.drop(columns=["ID", "CVSS Base Score", "CVSS Severity", "Description", "Config", "Reference"])
y = df["CVSS Base Score"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (optional but can help with training)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error')  


In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=2) 


Test

In [149]:
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")